In [ ]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import argparse
import yaml
import time
import datetime
import cv2
import numpy as np
import pandas as pd
import random

from skimage import io, transform
import matplotlib.pyplot as plt
from scipy.ndimage import zoom
from scipy import ndimage, misc

import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms, utils
from torchvision.utils import save_image
from torch.utils.data import Dataset, DataLoader
import torchvision
import torch
from torch.autograd import Variable
import torch.optim as optim
from sklearn.metrics import mean_absolute_error

from google.colab import drive
import os
import warnings
from collections import defaultdict
from sklearn.model_selection import KFold
import time
from scipy import stats
import sys
from sklearn.decomposition import PCA


In [ ]:
categorical_columns = ['TRAPTYPE', 'ATTRACTANTSUSED', 'SETTIMEOFDAY', 'COLLECTTIMEOFDAY', 'GENUS', 'SPECIES', 'TRAPID','TRAPSITE']
numerical_columns = ['LATITUDE', 'LONGITUDE', 'TRAPSET', 'YEAR','TRAPCOLLECT', 'DIFF_DAYS']

weather_columns = ['sunriseTime', 'sunsetTime', 'moonPhase', 
                     'precipIntensity', 'precipIntensityMax', 'precipProbability', 'temperatureHigh', 'temperatureHighTime',
                     'temperatureLow', 'temperatureLowTime', 'apparentTemperatureHigh', 'apparentTemperatureHighTime',
                     'apparentTemperatureLow', 'apparentTemperatureLowTime', 'dewPoint', 'humidity', 'pressure', 'windSpeed', 'windGust', 
                     'windGustTime', 'windBearing', 'cloudCover', 'uvIndex', 'uvIndexTime','visibility', 'temperatureMin',
                     'temperatureMinTime', 'temperatureMax', 'temperatureMaxTime', 'apparentTemperatureMin','apparentTemperatureMinTime', 'apparentTemperatureMax',
                     'apparentTemperatureMaxTime','time']

for i in range(1,15):
    for item in weather_columns:
        numerical_columns.append(item + str(i))

print(numerical_columns)

['LATITUDE', 'LONGITUDE', 'TRAPSET', 'YEAR', 'TRAPCOLLECT', 'DIFF_DAYS', 'sunriseTime1', 'sunsetTime1', 'moonPhase1', 'precipIntensity1', 'precipIntensityMax1', 'precipProbability1', 'temperatureHigh1', 'temperatureHighTime1', 'temperatureLow1', 'temperatureLowTime1', 'apparentTemperatureHigh1', 'apparentTemperatureHighTime1', 'apparentTemperatureLow1', 'apparentTemperatureLowTime1', 'dewPoint1', 'humidity1', 'pressure1', 'windSpeed1', 'windGust1', 'windGustTime1', 'windBearing1', 'cloudCover1', 'uvIndex1', 'uvIndexTime1', 'visibility1', 'temperatureMin1', 'temperatureMinTime1', 'temperatureMax1', 'temperatureMaxTime1', 'apparentTemperatureMin1', 'apparentTemperatureMinTime1', 'apparentTemperatureMax1', 'apparentTemperatureMaxTime1', 'time1', 'sunriseTime2', 'sunsetTime2', 'moonPhase2', 'precipIntensity2', 'precipIntensityMax2', 'precipProbability2', 'temperatureHigh2', 'temperatureHighTime2', 'temperatureLow2', 'temperatureLowTime2', 'apparentTemperatureHigh2', 'apparentTemperatureHig

In [ ]:
sentinel_frame = pd.read_csv('/content/drive/My Drive/Colab/Mosquito_Berkeley/Approach2/mosquito_weather_clean.csv')

sentinel_frame["TRAPSET"]= pd.to_datetime(sentinel_frame["TRAPSET"])
sentinel_frame["TRAPCOLLECT"]= pd.to_datetime(sentinel_frame["TRAPCOLLECT"])
sentinel_frame["DIFF_DAYS"] = (sentinel_frame["TRAPCOLLECT"] - sentinel_frame["TRAPSET"]).dt.days
sentinel_frame["TRAPSET"]= sentinel_frame["TRAPSET"].dt.week
sentinel_frame["TRAPCOLLECT"]= sentinel_frame["TRAPCOLLECT"].dt.week

for category in categorical_columns:
    sentinel_frame[category] = sentinel_frame[category].astype('category').cat.codes

for col in numerical_columns:
    sentinel_frame[col] = (sentinel_frame[col] - sentinel_frame[col].mean()) / (sentinel_frame[col].max() - sentinel_frame[col].min())

pca_frame = sentinel_frame[numerical_columns + categorical_columns]

def get_numerical_tensor(in_frame, numerical_columns):
    numerical_data = np.stack([in_frame[col].values for col in numerical_columns], 1)
    numerical_data = torch.tensor(numerical_data, dtype=torch.float)
    return numerical_data

pca = PCA(n_components=5)
principalComponents = pca.fit_transform(pca_frame)

numerical_columns = ['pc 1', 'pc 2','pc 3', 'pc 4','pc 5']

sentinel_frame_pca = pd.DataFrame(data = principalComponents 
             , columns = numerical_columns)

sentinel_frame_pca["TOTAL"] = sentinel_frame["TOTAL"]

In [ ]:
sentinel_frame_pca.columns

Index(['pc 1', 'pc 2', 'pc 3', 'pc 4', 'pc 5', 'TOTAL'], dtype='object')

In [ ]:
class Flatten(torch.nn.Module):
    def forward(self, x):
        return x.view(x.size()[0], -1)

class Swish(torch.autograd.Function):
    @staticmethod
    def forward(ctx, i):
        result = i * torch.sigmoid(i)
        ctx.save_for_backward(i)
        return result

    @staticmethod
    def backward(ctx, grad_output):
        i = ctx.saved_tensors[0]
        sigmoid_i = torch.sigmoid(i)
        return grad_output * (sigmoid_i * (1 + i * (1 - sigmoid_i)))
    
class SwishM(nn.Module):
    def forward(self, input_tensor):
        return Swish.apply(input_tensor)
        
def create_neural_modules(num_numerical_cols, output_size, hidden_size, dropout):
    all_layers = []
    input_size =  num_numerical_cols
    all_layers.append(nn.Linear(input_size, hidden_size))
    all_layers.append(nn.LeakyReLU())
    all_layers.append(nn.BatchNorm1d(hidden_size))
    all_layers.append(nn.Dropout(dropout))

    all_layers.append(nn.Linear(hidden_size, hidden_size))
    all_layers.append(nn.LeakyReLU())
    all_layers.append(nn.BatchNorm1d(hidden_size))
    all_layers.append(nn.Dropout(dropout))

    all_layers.append(nn.Linear(hidden_size, hidden_size))
    all_layers.append(nn.LeakyReLU())
    all_layers.append(nn.BatchNorm1d(hidden_size))
    all_layers.append(nn.Dropout(dropout))

    all_layers.append(nn.Linear(hidden_size, output_size))
    all_layers.append(nn.ReLU())

    return nn.Sequential(*all_layers)

class Sentinel_net(nn.Module):

    def __init__(self, num_numerical_cols, output_size, dropout, hidden_size):
        super(Sentinel_net, self).__init__()
        self.layers = create_neural_modules(num_numerical_cols, output_size, hidden_size, dropout)
        self.batch_norm_num = nn.BatchNorm1d(num_numerical_cols)

    def forward(self, x_numerical):
        y = self.layers(x_numerical)
        return y

In [ ]:
def train_shallow_net(train_n_t,test_n_t,outputs_train,outputs_test, dropout, hidden_size, lr):
    model = Sentinel_net(train_n_t.shape[1], 1, dropout, hidden_size)
    model.train().cuda()

    criterion = torch.nn.L1Loss()
    params_dict = dict(model.named_parameters())
    params = []
    best = sys.maxsize
    r_value = 0
    std_err = 0
    
    for key, value in params_dict.items():
        if 'weight' in key and 'layers' in key:
            params += [{'params':value, 'weight_decay':4e-3}]
        else:
            params += [{'params':value, 'weight_decay':0.0}]

    optimizer = torch.optim.RMSprop(params, lr=lr, alpha=0.99, eps=1e-08, weight_decay=0, momentum=0.9, centered=False)
    epoch = 0

    for i in range(1, 1000):
        y_pred = model(train_n_t)
        loss = criterion(y_pred.squeeze(), outputs_train)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        model.eval()
        with torch.no_grad():
            y_pred2 = model(test_n_t)
            c_out, c_pred = outputs_test.cpu().detach().numpy(), y_pred2.cpu().detach().numpy().squeeze()
            current_score = mean_absolute_error(c_out, c_pred)
            if current_score < best:
                best = current_score
                m, b, r_value, p_value, std_err = stats.linregress(c_out,c_pred)

            model.train()
    
    #print('MAE = ',current_score, 'r2 = ',r_value, 'SE = ',std_err)
    return best, r_value, std_err

In [ ]:
start_time = time.time()
learning = [0.01, 0.001] 
best_score_k_fold = sys.maxsize

print('3 Layers Architecture 5 PCA')

for dropout in range(1,5):
    dropout = round(dropout*.1, 2) 
    for hidden_size in range(50, 150, 10):
        for lr in learning: 

            scores, r_values, std_errors = [], [], []
            cv = KFold(n_splits=5, random_state=42, shuffle=True)
            foldc = 1
            for train_index, test_index in cv.split(sentinel_frame_pca, None):
                #print('Fold ', foldc, end =" ") 
                train, test  = sentinel_frame_pca.iloc[train_index], sentinel_frame_pca.iloc[test_index]
                train_n_t = get_numerical_tensor(train, numerical_columns).cuda()
                test_n_t = get_numerical_tensor(test, numerical_columns).cuda()

                outputs_train = torch.tensor(train['TOTAL'].values).flatten().type(torch.FloatTensor).cuda()
                outputs_test = torch.tensor(test['TOTAL'].values).flatten().type(torch.FloatTensor).cuda()

                best_fold_score, best_r_value, best_std_error = train_shallow_net(train_n_t,test_n_t,outputs_train,outputs_test, dropout, hidden_size, lr)
                scores.append(best_fold_score)
                r_values.append(best_r_value)
                std_errors.append(best_std_error)
                foldc += 1
            #print(scores)
            current_score = sum(scores) / len(scores)
            print('MAE = ', current_score ,'r2 = ', sum(r_values) / len(r_values) ,'SE = ', sum(std_errors) / len(std_errors),'lr = ', lr,'hz = ', hidden_size, 'dr = ', dropout )
            if current_score < best_score_k_fold:
                best_score_k_fold = current_score
                print('Best so far', best_score_k_fold)
                print('')


print("--- %s seconds ---" % (time.time() - start_time))

3 Layers Architecture 5 PCA
MAE =  9.733627319335938 r2 =  0.4034436986589263 SE =  0.013636154637773425 lr =  0.01 hz =  50 dr =  0.1
Best so far 9.733627319335938

MAE =  9.65397663116455 r2 =  0.45034574022990953 SE =  0.014628170307348398 lr =  0.001 hz =  50 dr =  0.1
Best so far 9.65397663116455

MAE =  9.739023590087891 r2 =  0.4129903390404307 SE =  0.013304764652519924 lr =  0.01 hz =  60 dr =  0.1
MAE =  9.753727531433105 r2 =  0.41849534323295706 SE =  0.014268685180691567 lr =  0.001 hz =  60 dr =  0.1
MAE =  9.716953468322753 r2 =  0.4146791698232703 SE =  0.013731628118271572 lr =  0.01 hz =  70 dr =  0.1
MAE =  9.752898216247559 r2 =  0.4491909112007374 SE =  0.014478314058350993 lr =  0.001 hz =  70 dr =  0.1
MAE =  9.747107696533202 r2 =  0.40118459556925073 SE =  0.014305376785470458 lr =  0.01 hz =  80 dr =  0.1
MAE =  9.698101043701172 r2 =  0.4326007704127555 SE =  0.014554383656331927 lr =  0.001 hz =  80 dr =  0.1
MAE =  9.740236854553222 r2 =  0.4010164908886555

In [ ]:
categorical_columns = ['TRAPTYPE', 'ATTRACTANTSUSED', 'SETTIMEOFDAY', 'COLLECTTIMEOFDAY', 'GENUS', 'SPECIES', 'TRAPID','TRAPSITE']
numerical_columns = ['LATITUDE', 'LONGITUDE', 'TRAPSET', 'YEAR','TRAPCOLLECT', 'DIFF_DAYS']

weather_columns = ['sunriseTime', 'sunsetTime', 'moonPhase', 
                     'precipIntensity', 'precipIntensityMax', 'precipProbability', 'temperatureHigh', 'temperatureHighTime',
                     'temperatureLow', 'temperatureLowTime', 'apparentTemperatureHigh', 'apparentTemperatureHighTime',
                     'apparentTemperatureLow', 'apparentTemperatureLowTime', 'dewPoint', 'humidity', 'pressure', 'windSpeed', 'windGust', 
                     'windGustTime', 'windBearing', 'cloudCover', 'uvIndex', 'uvIndexTime','visibility', 'temperatureMin',
                     'temperatureMinTime', 'temperatureMax', 'temperatureMaxTime', 'apparentTemperatureMin','apparentTemperatureMinTime', 'apparentTemperatureMax',
                     'apparentTemperatureMaxTime','time']

for i in range(1,15):
    for item in weather_columns:
        numerical_columns.append(item + str(i))

print(numerical_columns)

['LATITUDE', 'LONGITUDE', 'TRAPSET', 'YEAR', 'TRAPCOLLECT', 'DIFF_DAYS', 'sunriseTime1', 'sunsetTime1', 'moonPhase1', 'precipIntensity1', 'precipIntensityMax1', 'precipProbability1', 'temperatureHigh1', 'temperatureHighTime1', 'temperatureLow1', 'temperatureLowTime1', 'apparentTemperatureHigh1', 'apparentTemperatureHighTime1', 'apparentTemperatureLow1', 'apparentTemperatureLowTime1', 'dewPoint1', 'humidity1', 'pressure1', 'windSpeed1', 'windGust1', 'windGustTime1', 'windBearing1', 'cloudCover1', 'uvIndex1', 'uvIndexTime1', 'visibility1', 'temperatureMin1', 'temperatureMinTime1', 'temperatureMax1', 'temperatureMaxTime1', 'apparentTemperatureMin1', 'apparentTemperatureMinTime1', 'apparentTemperatureMax1', 'apparentTemperatureMaxTime1', 'time1', 'sunriseTime2', 'sunsetTime2', 'moonPhase2', 'precipIntensity2', 'precipIntensityMax2', 'precipProbability2', 'temperatureHigh2', 'temperatureHighTime2', 'temperatureLow2', 'temperatureLowTime2', 'apparentTemperatureHigh2', 'apparentTemperatureHig

In [ ]:
sentinel_frame = pd.read_csv('/content/drive/My Drive/Colab/Mosquito_Berkeley/Approach2/mosquito_weather_clean.csv')

sentinel_frame["TRAPSET"]= pd.to_datetime(sentinel_frame["TRAPSET"])
sentinel_frame["TRAPCOLLECT"]= pd.to_datetime(sentinel_frame["TRAPCOLLECT"])
sentinel_frame["DIFF_DAYS"] = (sentinel_frame["TRAPCOLLECT"] - sentinel_frame["TRAPSET"]).dt.days
sentinel_frame["TRAPSET"]= sentinel_frame["TRAPSET"].dt.week
sentinel_frame["TRAPCOLLECT"]= sentinel_frame["TRAPCOLLECT"].dt.week

for category in categorical_columns:
    sentinel_frame[category] = sentinel_frame[category].astype('category').cat.codes

for col in numerical_columns:
    sentinel_frame[col] = (sentinel_frame[col] - sentinel_frame[col].mean()) / (sentinel_frame[col].max() - sentinel_frame[col].min())

pca_frame = sentinel_frame[numerical_columns + categorical_columns]

def get_numerical_tensor(in_frame, numerical_columns):
    numerical_data = np.stack([in_frame[col].values for col in numerical_columns], 1)
    numerical_data = torch.tensor(numerical_data, dtype=torch.float)
    return numerical_data

pca = PCA(n_components=10)
principalComponents = pca.fit_transform(pca_frame)

numerical_columns = ['pc 1', 'pc 2','pc 3', 'pc 4','pc 5','pc 6', 'pc 7','pc 8', 'pc 9','pc 10']

sentinel_frame_pca = pd.DataFrame(data = principalComponents 
             , columns = numerical_columns)

sentinel_frame_pca["TOTAL"] = sentinel_frame["TOTAL"]

In [ ]:
start_time = time.time()
learning = [0.01, 0.001] 
best_score_k_fold = sys.maxsize

print('3 Layers Architecture 10 PCA')

for dropout in range(1,5):
    dropout = round(dropout*.1, 2) 
    for hidden_size in range(50, 150, 10):
        for lr in learning: 

            scores, r_values, std_errors = [], [], []
            cv = KFold(n_splits=5, random_state=42, shuffle=True)
            foldc = 1
            for train_index, test_index in cv.split(sentinel_frame_pca, None):
                #print('Fold ', foldc, end =" ") 
                train, test  = sentinel_frame_pca.iloc[train_index], sentinel_frame_pca.iloc[test_index]
                train_n_t = get_numerical_tensor(train, numerical_columns).cuda()
                test_n_t = get_numerical_tensor(test, numerical_columns).cuda()

                outputs_train = torch.tensor(train['TOTAL'].values).flatten().type(torch.FloatTensor).cuda()
                outputs_test = torch.tensor(test['TOTAL'].values).flatten().type(torch.FloatTensor).cuda()

                best_fold_score, best_r_value, best_std_error = train_shallow_net(train_n_t,test_n_t,outputs_train,outputs_test, dropout, hidden_size, lr)
                scores.append(best_fold_score)
                r_values.append(best_r_value)
                std_errors.append(best_std_error)
                foldc += 1
            #print(scores)
            current_score = sum(scores) / len(scores)
            print('MAE = ', current_score ,'r2 = ', sum(r_values) / len(r_values) ,'SE = ', sum(std_errors) / len(std_errors),'lr = ', lr,'hz = ', hidden_size, 'dr = ', dropout )
            if current_score < best_score_k_fold:
                best_score_k_fold = current_score
                print('Best so far', best_score_k_fold)
                print('')


print("--- %s seconds ---" % (time.time() - start_time))

3 Layers Architecture 10 PCA
MAE =  9.602915573120118 r2 =  0.4369563910960819 SE =  0.01468951380826089 lr =  0.01 hz =  50 dr =  0.1
Best so far 9.602915573120118

MAE =  9.700743675231934 r2 =  0.4168986325761793 SE =  0.011315830421636686 lr =  0.001 hz =  50 dr =  0.1
MAE =  9.720038604736327 r2 =  0.4220775881788582 SE =  0.015347464229858322 lr =  0.01 hz =  60 dr =  0.1
MAE =  9.676858520507812 r2 =  0.42877119433137967 SE =  0.015185309963452881 lr =  0.001 hz =  60 dr =  0.1
MAE =  9.567659950256347 r2 =  0.42034033080535094 SE =  0.0146006404750187 lr =  0.01 hz =  70 dr =  0.1
Best so far 9.567659950256347

MAE =  9.660308265686036 r2 =  0.4287495411476946 SE =  0.015954472930576533 lr =  0.001 hz =  70 dr =  0.1
MAE =  9.574982833862304 r2 =  0.4670576854100035 SE =  0.015534734963056837 lr =  0.01 hz =  80 dr =  0.1
MAE =  9.71222801208496 r2 =  0.4228005302805992 SE =  0.01415866303421554 lr =  0.001 hz =  80 dr =  0.1
MAE =  9.592313766479492 r2 =  0.4326924568401139 SE

In [ ]:
categorical_columns = ['TRAPTYPE', 'ATTRACTANTSUSED', 'SETTIMEOFDAY', 'COLLECTTIMEOFDAY', 'GENUS', 'SPECIES', 'TRAPID','TRAPSITE']
numerical_columns = ['LATITUDE', 'LONGITUDE', 'TRAPSET', 'YEAR','TRAPCOLLECT', 'DIFF_DAYS']

weather_columns = ['sunriseTime', 'sunsetTime', 'moonPhase', 
                     'precipIntensity', 'precipIntensityMax', 'precipProbability', 'temperatureHigh', 'temperatureHighTime',
                     'temperatureLow', 'temperatureLowTime', 'apparentTemperatureHigh', 'apparentTemperatureHighTime',
                     'apparentTemperatureLow', 'apparentTemperatureLowTime', 'dewPoint', 'humidity', 'pressure', 'windSpeed', 'windGust', 
                     'windGustTime', 'windBearing', 'cloudCover', 'uvIndex', 'uvIndexTime','visibility', 'temperatureMin',
                     'temperatureMinTime', 'temperatureMax', 'temperatureMaxTime', 'apparentTemperatureMin','apparentTemperatureMinTime', 'apparentTemperatureMax',
                     'apparentTemperatureMaxTime','time']

for i in range(1,15):
    for item in weather_columns:
        numerical_columns.append(item + str(i))

print(numerical_columns)

['LATITUDE', 'LONGITUDE', 'TRAPSET', 'YEAR', 'TRAPCOLLECT', 'DIFF_DAYS', 'sunriseTime1', 'sunsetTime1', 'moonPhase1', 'precipIntensity1', 'precipIntensityMax1', 'precipProbability1', 'temperatureHigh1', 'temperatureHighTime1', 'temperatureLow1', 'temperatureLowTime1', 'apparentTemperatureHigh1', 'apparentTemperatureHighTime1', 'apparentTemperatureLow1', 'apparentTemperatureLowTime1', 'dewPoint1', 'humidity1', 'pressure1', 'windSpeed1', 'windGust1', 'windGustTime1', 'windBearing1', 'cloudCover1', 'uvIndex1', 'uvIndexTime1', 'visibility1', 'temperatureMin1', 'temperatureMinTime1', 'temperatureMax1', 'temperatureMaxTime1', 'apparentTemperatureMin1', 'apparentTemperatureMinTime1', 'apparentTemperatureMax1', 'apparentTemperatureMaxTime1', 'time1', 'sunriseTime2', 'sunsetTime2', 'moonPhase2', 'precipIntensity2', 'precipIntensityMax2', 'precipProbability2', 'temperatureHigh2', 'temperatureHighTime2', 'temperatureLow2', 'temperatureLowTime2', 'apparentTemperatureHigh2', 'apparentTemperatureHig

In [ ]:
sentinel_frame = pd.read_csv('/content/drive/My Drive/Colab/Mosquito_Berkeley/Approach2/mosquito_weather_clean.csv')

sentinel_frame["TRAPSET"]= pd.to_datetime(sentinel_frame["TRAPSET"])
sentinel_frame["TRAPCOLLECT"]= pd.to_datetime(sentinel_frame["TRAPCOLLECT"])
sentinel_frame["DIFF_DAYS"] = (sentinel_frame["TRAPCOLLECT"] - sentinel_frame["TRAPSET"]).dt.days
sentinel_frame["TRAPSET"]= sentinel_frame["TRAPSET"].dt.week
sentinel_frame["TRAPCOLLECT"]= sentinel_frame["TRAPCOLLECT"].dt.week

for category in categorical_columns:
    sentinel_frame[category] = sentinel_frame[category].astype('category').cat.codes

for col in numerical_columns:
    sentinel_frame[col] = (sentinel_frame[col] - sentinel_frame[col].mean()) / (sentinel_frame[col].max() - sentinel_frame[col].min())

pca_frame = sentinel_frame[numerical_columns + categorical_columns]

def get_numerical_tensor(in_frame, numerical_columns):
    numerical_data = np.stack([in_frame[col].values for col in numerical_columns], 1)
    numerical_data = torch.tensor(numerical_data, dtype=torch.float)
    return numerical_data

pca = PCA(n_components=15)
principalComponents = pca.fit_transform(pca_frame)

numerical_columns = ['pc 1', 'pc 2','pc 3', 'pc 4','pc 5','pc 6', 'pc 7','pc 8', 'pc 9','pc 10','pc 11', 'pc 12','pc 13', 'pc 14','pc 15']

sentinel_frame_pca = pd.DataFrame(data = principalComponents 
             , columns = numerical_columns)

sentinel_frame_pca["TOTAL"] = sentinel_frame["TOTAL"]

In [ ]:
start_time = time.time()
learning = [0.01, 0.001] 
best_score_k_fold = sys.maxsize

print('3 Layers Architecture 15 PCA')

for dropout in range(1,5):
    dropout = round(dropout*.1, 2) 
    for hidden_size in range(50, 150, 10):
        for lr in learning: 

            scores, r_values, std_errors = [], [], []
            cv = KFold(n_splits=5, random_state=42, shuffle=True)
            foldc = 1
            for train_index, test_index in cv.split(sentinel_frame_pca, None):
                #print('Fold ', foldc, end =" ") 
                train, test  = sentinel_frame_pca.iloc[train_index], sentinel_frame_pca.iloc[test_index]
                train_n_t = get_numerical_tensor(train, numerical_columns).cuda()
                test_n_t = get_numerical_tensor(test, numerical_columns).cuda()

                outputs_train = torch.tensor(train['TOTAL'].values).flatten().type(torch.FloatTensor).cuda()
                outputs_test = torch.tensor(test['TOTAL'].values).flatten().type(torch.FloatTensor).cuda()

                best_fold_score, best_r_value, best_std_error = train_shallow_net(train_n_t,test_n_t,outputs_train,outputs_test, dropout, hidden_size, lr)
                scores.append(best_fold_score)
                r_values.append(best_r_value)
                std_errors.append(best_std_error)
                foldc += 1
            #print(scores)
            current_score = sum(scores) / len(scores)
            print('MAE = ', current_score ,'r2 = ', sum(r_values) / len(r_values) ,'SE = ', sum(std_errors) / len(std_errors),'lr = ', lr,'hz = ', hidden_size, 'dr = ', dropout )
            if current_score < best_score_k_fold:
                best_score_k_fold = current_score
                print('Best so far', best_score_k_fold)
                print('')


print("--- %s seconds ---" % (time.time() - start_time))

3 Layers Architecture 15 PCA
MAE =  9.646173858642578 r2 =  0.4173961770997465 SE =  0.015701952789731695 lr =  0.01 hz =  50 dr =  0.1
Best so far 9.646173858642578

MAE =  9.743864631652832 r2 =  0.46842787607315717 SE =  0.01469851082376265 lr =  0.001 hz =  50 dr =  0.1
MAE =  9.61083526611328 r2 =  0.4330567815140087 SE =  0.01704142011464425 lr =  0.01 hz =  60 dr =  0.1
Best so far 9.61083526611328

MAE =  9.76803150177002 r2 =  0.4438574690861562 SE =  0.014539557424832946 lr =  0.001 hz =  60 dr =  0.1
MAE =  9.670787620544434 r2 =  0.4085196931827201 SE =  0.013749229863535842 lr =  0.01 hz =  70 dr =  0.1
MAE =  9.693304634094238 r2 =  0.4778965594044317 SE =  0.014223249186143126 lr =  0.001 hz =  70 dr =  0.1
MAE =  9.670381927490235 r2 =  0.4151036339993596 SE =  0.015021814882961749 lr =  0.01 hz =  80 dr =  0.1
MAE =  9.740903091430663 r2 =  0.4130994161478748 SE =  0.01385544155935787 lr =  0.001 hz =  80 dr =  0.1
MAE =  9.698388671875 r2 =  0.4231899269071726 SE =  0